# 005: Cross Validate Models

In [2]:
import os
import sys

import numpy as np

sys.path.append("../")
from models import OrdinaryLeastSquares, LogisticRegression, LinearSVM, KNearestNeighbors
from model_selection import cross_validation
from preprocessing import preprocess

%load_ext autoreload
%autoreload 2

def print_results(cv_results):
    print(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}")
    print(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}")
    print(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}")
    print(f" & {np.mean(cv_results.f1_scores)*100:.1f}±{np.std(cv_results.f1_scores)*100:.1f} & "
          f"{np.mean(cv_results.f2_scores)*100:.1f}±{np.std(cv_results.f2_scores)*100:.1f} & "
          f"{np.mean(cv_results.auc_rocs)*100:.1f}±{np.std(cv_results.auc_rocs)*100:.1f}")

In [3]:
x_train, _, y_train, _ = preprocess(one_hot_encoding=False, save_dir="../data/dataset_prep")
x_train.shape

Loading raw data...
Replacing missing value codes with np.nan...
Saving preprocessed data to ../data/dataset_prep...


(328135, 321)

In [22]:
train = np.load("../data/dataset_prep/train.npz")
x_train, y_train = train["x_train"], train["y_train"]
x_train.shape

(328135, 321)

In [1]:
model_settings = [
    #{"model_class": OrdinaryLeastSquares},
    #{"model_class": LogisticRegression, "gamma": 1e-2},
    {"model_class": LinearSVM},
    #{"model_class": KNearestNeighbors, "use_pca": True},
]
for model in model_settings:
    print(f"Cross-validating model: {model['model_class'].__name__}")
    num_samples = int(1e4) if model['model_class'] != KNearestNeighbors else int(1e4)
    cv_results = cross_validation(x_train[:num_samples], y_train[:num_samples], verbose=True, **model)
    print_results(cv_results)
    break
    out_dir = "../results"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{model['model_class'].__name__}.txt")

    with open(out_path, "w") as f:
        f.write(f"Model: {model['model_class'].__name__}\n\n")
        f.write("cv_results repr:\n")
        f.write(repr(cv_results))
        f.write("\n\n")
        f.write("Aggregated metrics:\n")
        f.write(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}\n")
        f.write(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}\n")
        f.write(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}\n")
    

NameError: name 'LinearSVM' is not defined

In [8]:
print_results(cv_results)

Average F1-score: 42.8% ± 0.3
Average F2-score: 50.1% ± 0.6
Average AUC-ROC: 86.1% ± 0.1
 & 42.8±0.3 & 50.1±0.6 & 86.1±0.1
